In [ ]:
import sys
sys.path.append("../")
from mir.common import RepertoireDataset, VDJtoolsParser
from mir.basic import FrequencyTable, DiversityIndices, RarefactionCurve
import pandas as pd
from plotnine import *

Load metadata

In [ ]:
metadata = pd.read_table('assets/samples/aging_3year/metadata.txt', sep='\t')
metadata

Load four repertoires for two donors taken 3 years apart

In [ ]:
dataset = RepertoireDataset.load(parser=VDJtoolsParser(), 
                                 metadata=metadata,
                                 paths=['assets/samples/aging_3year/' + r['file_name'] 
                                        for _, r in metadata.iterrows()])
print(dataset)

Compute clonotype frequency tables

In [ ]:
freq_table = FrequencyTable.from_repertoire(dataset[0])
print(freq_table)
print(DiversityIndices(freq_table))
print(RarefactionCurve(freq_table))

Diversity index table

In [ ]:
DiversityIndices.for_dataset(dataset)

Hill curve plot

In [ ]:
df_hc = DiversityIndices.hill_curve_for_dataset(dataset)
df_hc['age'] = df_hc['age'].apply(lambda x: '+3years' if x not in {27, 47} else 'baseline')
plt = (ggplot(df_hc, aes(x='q', y='H_q', 
                         color='donor', linetype='age')) + 
       geom_vline(xintercept=1., color='lightgrey') +
       geom_line() + 
       scale_x_log10(name='q') +
       scale_y_log10(name='H_q') +
       scale_color_manual(values=['blue', 'red'],
                          labels=['young', 'old']) +
       theme_classic())
print(plt) 

Rarefaction curves

In [ ]:
df_rarefy = RarefactionCurve.for_dataset(dataset)
df_rarefy['age'] = df_rarefy['age'].apply(lambda x: '+3years' if x not in {27, 47} else 'baseline')
plt = (ggplot(df_rarefy, aes(group = 'file_name', x='count_star', y='species')) +         
       geom_ribbon(aes(ymin='species - 1.96 * var_species ** 0.5', 
                       ymax='species + 1.96 * var_species ** 0.5'), alpha=0.25) +
       geom_line(aes(color='donor', linetype='age')) + 
       scale_color_manual(values=['blue', 'red'],
                          labels=['young', 'old']) +
       theme_classic())
print(plt) 